In [45]:
#reference: https://dev.to/thepylot/compare-documents-similarity-using-python-nlp-4odp 
import gensim
from nltk.tokenize import word_tokenize, sent_tokenize
import json
import numpy as np
f = open('./json_files/label3_to_label2.json')
l3_l2 = json.load(f)
our_compared_list_l2 = set()
our_compared_list_l3 = set()
for k, v in l3_l2.items():
    our_compared_list_l2.add(v)
    our_compared_list_l3.add(k)
our_compared_list_l2 = list(our_compared_list_l2)
our_compared_list_l3 = list(our_compared_list_l3)

#Tokenize words and create dictionary
gen_docs = [[w.lower() for w in word_tokenize(text)] for text in our_compared_list_l2]
dictionary = gensim.corpora.Dictionary(gen_docs)

gen_docs_3 = [[w.lower() for w in word_tokenize(text)] for text in our_compared_list_l3]
dictionary_3 = gensim.corpora.Dictionary(gen_docs_3)
#print(dictionary.token2id)

#create a bad of words (Corpus)
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
corpus_3 = [dictionary_3.doc2bow(gen_doc) for gen_doc in gen_docs_3]

#print(corpus_3)
tf_idf = gensim.models.TfidfModel(corpus)
tf_idf_3 = gensim.models.TfidfModel(corpus=corpus_3)

for doc in tf_idf[corpus]:
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc]) 

for doc in tf_idf_3[corpus_3]:
    print([[dictionary_3[id], np.around(freq, decimals=2)] for id, freq in doc])




[['short', 1.0]]
[['lace', 0.71], ['shoe', 0.71]]
[['cloak', 1.0]]
[['bath', 0.71], ['robe', 0.71]]
[['bandana', 1.0]]
[['boot', 1.0]]
[['wallet', 1.0]]
[['skirts', 1.0]]
[['leather', 0.71], ['suede', 0.71]]
[['perfume', 1.0]]
[['sandal', 1.0]]
[['activewear', 1.0]]
[['slipper', 1.0]]
[['jewelry', 1.0]]
[['lingerie', 1.0]]
[['shirts', 1.0]]
[['sock', 1.0]]
[['belt', 1.0]]
[['homeware', 1.0]]
[['leggings', 1.0]]
[['sticker', 1.0]]
[['care', 0.59], ['hair', 0.81]]
[['care', 0.59], ['nail', 0.81]]
[['pocket', 0.71], ['square', 0.71]]
[['others', 1.0]]
[['tight', 1.0]]
[['blazer/suit', 1.0]]
[['tank', 0.71], ['tops', 0.71]]
[['watch', 1.0]]
[['pants/trousers', 1.0]]
[['underwear', 1.0]]
[['bag', 1.0]]
[['sweaters', 1.0]]
[['dresses', 1.0]]
[['swimwear', 1.0]]
[['sleepwear', 1.0]]
[['jumpsuits', 1.0]]
[['skateboard', 1.0]]
[['headwear', 1.0]]
[['beauty', 0.71], ['tools', 0.71]]
[['coats/jackets', 1.0]]
[['glove', 1.0]]
[['ties', 1.0]]
[['sleeve', 1.0]]
[['clip', 1.0]]
[['scarf', 1.0]]
[['co

In [34]:
gen_docs[33]

['dresses']

In [54]:
# building the index
sims_3 = gensim.similarities.Similarity('./',tf_idf_3[corpus_3],num_features=len(dictionary_3))
sims = gensim.similarities.Similarity('./1',tf_idf[corpus],num_features=len(dictionary))
#Create Query Document


In [58]:
import pandas as pd
products = pd.read_csv("./temp.csv")
for i in range(0, products.shape[0]):
    pt = products.loc[i]['product_type']

    query_doc = [w.lower() for w in word_tokenize(pt)]
    query_doc_bow = dictionary.doc2bow(query_doc)

    query_doc_tf_idf = tf_idf[query_doc_bow]
    #print("Comparing Result: ", sims[query_doc_bow].tolist())
    res = sims[query_doc_bow].tolist()

    keys = [''.join(gen_docs[i]) for i in range(len(res))]
    values = [res[i] for i in range(len(res))]
    l2_distance = dict(zip(keys, values))
    sorted_l2_distance = list(dict(sorted(l2_distance.items(), key=lambda item: item[1], reverse=True)))
    
    products.loc[i, 'nltk_tfidf_cos_similarity_layer2'] = sorted_l2_distance[0]

    query_doc_3 = [w.lower() for w in word_tokenize(pt)]
    query_doc_bow_3 = dictionary_3.doc2bow(query_doc_3)

    query_doc_tf_idf_3 = tf_idf_3[query_doc_bow_3]
    #print("Comparing Result: ", sims[query_doc_bow].tolist())
    res_3 = sims_3[query_doc_bow_3].tolist()

    keys = [''.join(gen_docs_3[i]) for i in range(len(res_3))]
    values = [res_3[i] for i in range(len(res_3))]
    l3_distance = dict(zip(keys, values))
    sorted_l3_distance = list(dict(sorted(l3_distance.items(), key=lambda item: item[1], reverse=True)))

    products.loc[i, 'nltk_tfidf_cos_similarity_layer3'] = sorted_l3_distance[0]




,Unnamed: 0,#,product_type,Spacy_similarity_layer2,Spacy_similarity_layer3,nltk_tfidf_cos_similarity_layer3,nltk_tfidf_cos_similarity_layer2,product_type(modified),main_category,sub_category,match_80,match_60,label_1st,label_2nd,label_3rd
0,0,1,Bikini Bottom,Tank Tops,bikini bottom,bikinibottom,short,bikini bottom,other_clothing,bikini bottom,bikini bottom,bikini bottom,4,Swimwear,Bikini Bottom
1,1,2,Bikini Top,Tank Tops,bikini top,bikinitop,short,bikini top,other_clothing,bikini top,bikini top,bikini,4,Swimwear,Bikini Top
2,2,3,Homeware - Selfcare,Homeware,semi-slim pant,faceserum,homeware,homeware selfcare,homeware,selfcare,NaN,NaN,7,NaN,NaN
3,3,4,accessories,Leggings,leather jackets,faceserum,short,accessories,accessories,NaN,NaN,NaN,6,NaN,NaN
4,4,5,Cover up,Tight,cover up,coverup,short,cover up,NaN,cover up,cover up,cover up,4,Swimwear,Cover Up


In [77]:
products.to_csv("temp.csv")

In [11]:


query_doc = [w.lower() for w in word_tokenize(text)]
query_doc_bow = dictionary.doc2bow(query_doc)

In [31]:
query_doc_tf_idf = tf_idf[query_doc_bow]
print("Comparing Result: ", sims[query_doc_bow].tolist())
res = sims[query_doc_bow].tolist()
len(res)

Comparing Result:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


52

In [39]:
keys = [''.join(gen_docs[i]) for i in range(len(res))]
values = [res[i] for i in range(len(res))]
l2_distance = dict(zip( keys, values))
sorted_l2_distance = list(dict(sorted(l2_distance.items(), key=lambda item: item[1], reverse=True)))
print(sorted_l2_distance[0])
#get the most similar item

dresses
